# Model_v3 with pure text

## Generate t2t data

In [2]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems

In [3]:
# import generate_problem # self-defined 'generate_problem.py' file in the same folder.

# # define the directory for generated data
# USR_DIR= './'
# PROBLEM = 'OnlineRevewProjectUSYD_cleaned'#_v1
# TMP_DIR = './tmp' # Where data files from internet stored
# DATA_LOC = './data' # Where pre-prcessed data is stored

# # Generated training data
# t2t_problem = generate_problem.OnlineRevewProjectUSYD(PROBLEM)
# t2t_problem.generate_data(DATA_LOC, TMP_DIR) 

INFO:tensorflow:Found vocab file: ./data/vocab.online_revew_project_usyd.32768.subwords
INFO:tensorflow:Skipping generator because outputs files exists at ['./data/online_revew_project_usyd-unshuffled-train-00000-of-00009', './data/online_revew_project_usyd-unshuffled-train-00001-of-00009', './data/online_revew_project_usyd-unshuffled-train-00002-of-00009', './data/online_revew_project_usyd-unshuffled-train-00003-of-00009', './data/online_revew_project_usyd-unshuffled-train-00004-of-00009', './data/online_revew_project_usyd-unshuffled-train-00005-of-00009', './data/online_revew_project_usyd-unshuffled-train-00006-of-00009', './data/online_revew_project_usyd-unshuffled-train-00007-of-00009', './data/online_revew_project_usyd-unshuffled-train-00008-of-00009', './data/online_revew_project_usyd-unshuffled-dev-00000-of-00001']
INFO:tensorflow:Skipping shuffle because output files exist


In [ ]:
import generate_problem # self-defined 'generate_problem.py' file in the same folder.

# define the directory for generated data
USR_DIR= './'
PROBLEM = 'OnlineRevewProjectUSYD_cleaned'#_v1
TMP_DIR = './tmp' # Where data files from internet stored
DATA_LOC = './data' # Where pre-prcessed data is stored

# Generated training data
t2t_problem = generate_problem.OnlineRevewProjectUSYD(PROBLEM)
t2t_problem.generate_data(DATA_LOC, TMP_DIR) 

In [12]:
# select the transformer model
from tensor2tensor.utils import registry
from tensor2tensor import models

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files'

## Define Hyper params and Train model

In [13]:
from tensor2tensor.utils.trainer_lib import create_hparams
import json
# Init Hparams object from T2T Problem
hparams = create_hparams(HPARAMS)
hparams.batch_size = 32 #512 
# hparams.learning_rate_warmup_steps = 45000
hparams.learning_rate = 0.2 # default 0.2
# hparams.num_encoder_layers = 6 # default 0
# hparams.num_decoder_layers = 6 # default 0
# hparams.attention_dropout= 0.1 #default
# hparams.num_hidden_layers = 6 #default # can try 12
# hparams.max_input_seq_length = 512
# hparams.max_target_seq_length = 

# print out all Hparams
print(json.loads(hparams.to_json()))

{'ffn_layer': 'dense_relu_dense', 'optimizer_multistep_accumulate_steps': None, 'learning_rate_minimum': None, 'eval_run_autoregressive': False, 'layer_postprocess_sequence': 'da', 'multiproblem_fixed_train_length': -1, 'use_fixed_batch_size': False, 'mlperf_mode': False, 'kernel_width': 1, 'summarize_vars': False, 'attention_key_channels': 0, 'vocab_divisor': 1, 'weight_noise': 0.0, 'warm_start_from_second': '', 'moe_overhead_train': 1.0, 'batch_shuffle_size': 512, 'norm_type': 'layer', 'weight_dtype': 'float32', 'multiproblem_max_target_length': -1, 'shared_embedding': False, 'learning_rate_decay_steps': 5000, 'self_attention_type': 'dot_product', 'label_smoothing': 0.1, 'learning_rate_decay_rate': 1.0, 'tpu_enable_host_call': False, 'moe_hidden_sizes': '2048', 'relu_dropout_broadcast_dims': '', 'kernel_height': 3, 'daisy_chain_variables': True, 'modality': {}, 'num_hidden_layers': 6, 'max_input_seq_length': 0, 'compress_steps': 0, 'sampling_method': 'argmax', 'pad_batch': False, 'sc

In [14]:
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
# running time calculation
import timeit
start = timeit.default_timer()

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)

# create model experiment
t2t_model = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=t2t_problem, 
        data_dir='./data', 
        train_steps=40001, 
        eval_steps=200 
    )

# training
t2t_model.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_eval_distribute': None, '_save_checkpoints_steps': 1000, '_task_type': None, '_tf_random_seed': None, '_task_id': 0, '_evaluation_master': '', 't2t_device_info': {'num_async_replicas': 1}, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 20, '_save_summary_steps': 100, '_model_dir': 'model_files', '_num_ps_replicas': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8c99a45470>, '_log_step_count_steps': 100, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_num_worker_re

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_45085_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_45085_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_45085_512.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 67203584
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model_files/model.ckpt-1000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1000 into model_files/model.ckpt.
INFO:tensorflow:loss = 0.818578, step = 1000
INFO:tensorflow:global_step/sec: 1.4024
INFO:tensorflow:loss = 3.3732483, step = 1100 (71.310 sec)
INFO:tensorflow:global_step/sec: 1.43091
INFO:tensorflow:loss = 2.1069698, step = 1200 (69.885 sec)
INFO:tensorflow:global_step/sec: 1.42606
INFO:tensorflow:loss = 4.095603, step = 1300 (70.123 sec)
INFO:tensorflow:global_step/sec: 1.44588
INFO:tensorflow:loss = 3.2072134, step = 1400 (69.162 sec)
INFO:tensorflow:global_step/sec: 1.37537
INFO:tensorfl

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_45085_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_45085_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_45085_512.top
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.n

UnknownError: model_files/model.ckpt-3000_temp_d80d52441a3545a185fb2c9e8a28bc23/part-00001-of-00002.data-00000-of-00001.tempstate8449577402577481212; Input/output error
	 [[node save/SaveV2_1 (defined at /usr/local/lib/python3.5/dist-packages/tensorflow_estimator/python/estimator/estimator.py:1403) ]]

Caused by op 'save/SaveV2_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.

In [10]:
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
# running time calculation
import timeit
start = timeit.default_timer()

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)

# create model experiment
t2t_model = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=t2t_problem, #PROBLEM,
        data_dir='./data', 
        train_steps=40001,
        eval_steps=200 
    )

# training
t2t_model.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_eval_distribute': None, '_save_checkpoints_steps': 1000, '_task_type': None, '_tf_random_seed': None, '_task_id': 0, '_evaluation_master': '', 't2t_device_info': {'num_async_replicas': 1}, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 20, '_save_summary_steps': 100, '_model_dir': 'model_files', '_num_ps_replicas': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8d68158cc0>, '_log_step_count_steps': 100, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_num_worker_re

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_45085_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_45085_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_45085_512.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 67203584
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into model_files/model.ckpt.
INFO:tensorflow:loss = 9.735924, step = 0
INFO:tensorflow:global_step/sec: 1.47582
INFO:tensorflow:loss = 7.941203, step = 100 (67.760 sec)
INFO:tensorflow:global_step/sec: 1.34412
INFO:tensorflow:loss = 7.3405256, step = 200 (74.400 sec)
INFO:tensorflow:global_step/sec: 1.467
INFO:tensorflow:loss = 5.628508, step = 300 (68.166 sec)
INFO:tensorflow:global_step/sec: 1.35291
INFO:tensorflow:loss = 5.9737744, step = 400 (73.915 sec)
INFO:tensorflow:global_step/sec: 1.42609
INFO:tensorflow:loss = 5.5982184, step = 500 (70.121 sec)
INFO:tensorflow:global_step/sec: 1.32692
INFO:tensorflow:loss = 5.0106006, step = 600 (75.363 sec)
INFO:tensorflow:global_step/sec: 1.51763
INFO:tensorflow:loss = 5.014288, step = 700 (65.891 se

UnknownError: model_files/model.ckpt-2000_temp_df46dfdffe1d4f36ae72540283899649/part-00001-of-00002.data-00000-of-00001.tempstate6800881703259272977; Input/output error
	 [[node save/SaveV2_1 (defined at /usr/local/lib/python3.5/dist-packages/tensorflow_estimator/python/estimator/estimator.py:1403) ]]

Caused by op 'save/SaveV2_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.

In [13]:
# from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
# # running time calculation
# import timeit
# start = timeit.default_timer()

# RUN_CONFIG = create_run_config(
#     model_name=MODEL,
#     model_dir=TRAIN_DIR,
#     save_checkpoints_steps = 2000)

# # create model experiment
# t2t_model = create_experiment(
#         run_config=RUN_CONFIG,
#         hparams=hparams,
#         model_name=MODEL,
#         problem_name=t2t_problem, #PROBLEM,
# #         t2t_usr_dir='./',
#         data_dir='./data', 
#         train_steps=40001, # Total number of train steps for all Epochs
#         eval_steps=200 # Number of steps to perform for each evaluation
#     )

# # training
# t2t_model.train_and_evaluate()

# # running time check (running on 1GPU server)
# stop = timeit.default_timer()
# print('Time: {} s'.format(round(stop - start,2)))

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'use_tpu': False, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_num_ps_replicas': 0, '_device_fn': None, 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7fbcd355a668>, 't2t_device_info': {'num_async_replicas': 1}, '_model_dir': 'model_files', '_save_checkpoints_steps': 10000, '_task_id': 0, '_task_type': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec ob

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_45085_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_45085_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_45085_512.top
INFO:tensorflow:Base learning rate: 0.100000
INFO:tensorflow:Trainable Variables Total size: 67203584
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into model_files/model.ckpt.
INFO:tensorflow:loss = 9.871942, step = 0
INFO:tensorflow:global_step/sec: 1.35857
INFO:tensorflow:loss = 8.675489, step = 100 (73.608 sec)
INFO:tensorflow:global_step/sec: 1.48466
INFO:tensorflow:loss = 7.162033, step = 200 (67.357 sec)
INFO:tensorflow:global_step/sec: 1.28675
INFO:tensorflow:loss = 6.9941096, step = 300 (77.714 sec)
INFO:tensorflow:global_step/sec: 1.48497
INFO:tensorflow:loss = 6.3784957, step = 400 (67.342 sec)
INFO:tensorflow:global_step/sec: 1.35022
INFO:tensorflow:loss = 6.342918, step = 500 (74.062 sec)
INFO:tensorflow:global_step/sec: 1.4502
INFO:tensorflow:loss = 6.2964, step = 600 (68.956 sec)
INFO:tensorflow:global_step/sec

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_45085_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_45085_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_45085_512.top
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.n

INFO:tensorflow:loss = 0.23409516, step = 6900 (67.295 sec)
INFO:tensorflow:global_step/sec: 1.37857
INFO:tensorflow:loss = 1.3663039, step = 7000 (72.540 sec)
INFO:tensorflow:global_step/sec: 1.472
INFO:tensorflow:loss = 1.1142291, step = 7100 (67.936 sec)
INFO:tensorflow:global_step/sec: 1.38404
INFO:tensorflow:loss = 0.68717486, step = 7200 (72.252 sec)
INFO:tensorflow:global_step/sec: 1.50912
INFO:tensorflow:loss = 0.45339054, step = 7300 (66.263 sec)
INFO:tensorflow:global_step/sec: 1.34152
INFO:tensorflow:loss = 0.20476814, step = 7400 (74.543 sec)
INFO:tensorflow:global_step/sec: 1.5196
INFO:tensorflow:loss = 1.1575347, step = 7500 (65.807 sec)
INFO:tensorflow:global_step/sec: 1.36073
INFO:tensorflow:loss = 0.35480994, step = 7600 (73.490 sec)
INFO:tensorflow:global_step/sec: 1.46776
INFO:tensorflow:loss = 0.5912126, step = 7700 (68.131 sec)
INFO:tensorflow:global_step/sec: 1.36159
INFO:tensorflow:loss = 0.68429184, step = 7800 (73.444 sec)
INFO:tensorflow:global_step/sec: 1.563

UnknownError: model_files/model.ckpt-10000_temp_1e28b76b124449588f7b1eb313bb73f4/part-00001-of-00002.data-00000-of-00001.tempstate12549872060078745595; Input/output error
	 [[node save/SaveV2_1 (defined at /usr/local/lib/python3.5/dist-packages/tensorflow_estimator/python/estimator/estimator.py:1403) ]]

Caused by op 'save/SaveV2_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 132, in start
    sel